# Setting up environment

In [8]:
import pickle
import numpy as np
import scipy
import scipy.io as sio
import sys
sys.path.append('..')

In [9]:
data_prefix = ''

# Getting the Data from a Predefined Matrix [Scraped a while ago by Svebor]

Labels is a list showing which person corresponds to which image [Note: there are some people that show up multiple times]

In [10]:
# how to load ground truth labels
gt_name = 'LFW_all_subjects_id.mat'
gt_file_full = data_prefix + gt_name
gt_mat = sio.loadmat(gt_file_full)
labels = np.squeeze(gt_mat['all_subjects_ids'])
print labels

[   0    1    2 ..., 5746 5747 5748]


There are 5748 People in this LFW dataset

# Loading 512 bit long hashcodes that were generated from the Neural Network

In [11]:
# how to load hashcodes
hashcodes_name = 'LFW_hash512_randomcrop_prelu_newproto_rgb_notV7.3.mat'
hashcodes_file_full = data_prefix + hashcodes_name
hashcodes_mat = sio.loadmat(hashcodes_file_full)
#print hashcodes_mat['templates']
all_templates = hashcodes_mat['templates']
# all templates contains: subject_name, subject_id, file, features (actually hashcodes in this case)
nb_samples = hashcodes_mat['templates'].shape[1]
all_samples_tmp = []
all_samples_files = []
for i in range(nb_samples):
    all_samples_tmp.append(np.squeeze(all_templates[0,i][3]))
    all_samples_files.append(str(np.squeeze(all_templates[0,i][2])))
all_samples = np.asarray(all_samples_tmp)

In [12]:
print all_samples_files[:10]

['AJ_Cook_0001.jpg', 'AJ_Lamas_0001.jpg', 'Aaron_Eckhart_0001.jpg', 'Aaron_Guiel_0001.jpg', 'Aaron_Patterson_0001.jpg', 'Aaron_Peirsol_0001.jpg', 'Aaron_Peirsol_0002.jpg', 'Aaron_Peirsol_0003.jpg', 'Aaron_Peirsol_0004.jpg', 'Aaron_Pena_0001.jpg']


In [13]:
print all_samples[:2,:]
# hash codes should actually be binarized.
# Check what is the best way to represent binary codes in python
print (all_samples>0.5)[:2,:]
print all_samples.shape

[[  9.99719501e-01   9.99999583e-01   9.92673457e-01 ...,   1.00000000e+00
    1.00000000e+00   1.06802311e-07]
 [  6.81201158e-08   3.98614723e-03   1.00000000e+00 ...,   9.99999821e-01
    2.63930566e-12   7.86280156e-12]]
[[ True  True  True ...,  True  True False]
 [False False  True ...,  True False False]]
(13233, 512)


# Getting distances between each hashcode

In [14]:
# how to load precomputed distance matrix
fulldist_name = 'LFW_full_hamming_dist.mat'
fulldist_file = data_prefix + fulldist_name
fulldist = sio.loadmat(fulldist_file)

In [15]:
fulldist_np = fulldist['full_hamming_dist']

In [16]:
fulldist_np.shape

(13233, 13233)

# Scraping LFW dataset from CAVE

In [17]:
f = open('lfw_attributes.txt', 'r')

f.readline() # Header
f.readline() # Attributes List

labels = []
data = []
count = 0
for line in f:
    count += 1
    line = line.split("\n")[0]
    arr = line.split("	")
    label = arr[0] 
    features = []
    for elem in arr[2:]:
        if float(elem) <= 0:
            features.append(0)
        else:
            features.append(1)
    retVal = []
    for elem in label.split():
        retVal.append(elem)
    labels.append("_".join(retVal) + "_" + str(arr[1]).zfill(4) + ".jpg")
    data.append(np.array(features))
print len(labels)
print str(count)
data = np.array(data)

13143
13143


In [18]:
print type(data)

<type 'numpy.ndarray'>


In [19]:
# print all_samples_files[2]
# print labels[0]
# print labels[0] in all_samples_files[2]

print len(labels)
print len(all_samples_files)
labels_count = 0
hash_count = 0
count = 0

dictionary = {}
for elem in labels:
    dictionary[elem] = True
count = 0
colDel = []
for elem in all_samples_files:
    if elem not in dictionary:
        colDel.append(count)
    count += 1

print '---'
print len(labels)
print len(all_samples_files)

13143
13233
---
13143
13233


In [20]:
print len(colDel)

count = len(colDel) - 1
while count >= 0:
    fulldist_np = scipy.delete(fulldist_np, colDel[count], 0)  # delete second row of A
    fulldist_np = scipy.delete(fulldist_np, colDel[count], 1)  # delete second column of C
    count = count - 1
    

90


In [21]:
print len(labels)
print len(fulldist_np[0])
print fulldist_np.shape

13143
13143
(13143, 13143)


In [22]:
print len(data[2])

73


In [23]:
from scipy.spatial import distance
data_dist = distance.cdist(data, data, 'hamming')

In [24]:
type(data_dist)
data_dist[0][100]

0.12328767123287671

In [25]:
normalized_hash_dist= fulldist_np/512.0

In [26]:
print normalized_hash_dist[2000][1300]
print fulldist_np[2000][1300]

0.431640625
221


In [27]:
print data_dist[6][5]

0.232876712329


In [28]:
print type(data_dist)
print data_dist.shape
print type(normalized_hash_dist)
print normalized_hash_dist.shape

<type 'numpy.ndarray'>
(13143, 13143)
<type 'numpy.ndarray'>
(13143, 13143)


In [29]:
x1 = np.array([1,2,3])
x2 = np.array([4,5,6])
x3 = []
x3.append(x1)
x3.append(x2)
x3 = np.array(x3)


x11 = np.array([11,12,13])
x12 = np.array([14,15,16])
x13 = []
x13.append(x11)
x13.append(x12)
x13 = np.array(x13)

print np.subtract(x3, x13)

[[-10 -10 -10]
 [-10 -10 -10]]


In [ ]:
np.subtract(normalized_hash_dist, fulldist_np)

from scipy.spatial import distance
hash_dist = distance.cdist(all_samples, all_samples, 'euclidean')

In [80]:
print len(hash_dist)


13233


In [81]:
print len(hash_dist[0])

13233


In [94]:
print labels[0:50]




[ 0  1  2  3  4  5  5  5  5  6  7  7  8  9 10 11 12 13 13 14 14 14 14 15 16
 17 18 18 18 18 19 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20]


In [97]:
print hash_dist[37][36]
print hash_dist[37][38]
print hash_dist[36][38]
print hash_dist[35][39]
print hash_dist[39][40]
print hash_dist[40][35]

8.1039511579
9.2625290269
10.0453316283
11.301439035
8.92844945144
10.0961797268


In [99]:
best = 0
for elem in hash_dist:
    best = max(best, max(elem))

In [122]:
best2 = 0
for elem in fulldist_np:
    best2 = max(best2, max(elem))
    

In [123]:
best2

387

In [ ]:
dist_np = fulldist_np / (best2 / 100.0)
has_np = hash_dist / (best / 100.0)


In [140]:
print max(dist_np[100])
print max(fulldist_np[100])


92.7648578811
359
